Dla danego szeregowania należy wyliczyć wartość kary oraz optymalna kolejność zadań

Dane skladaja sie z trzech tabel
- czas produkcji
- waga (wysokość kary)
- pozadany czas produkcji

### LAB 4

Obliczanie wartości kary na podstawie zadanej kolejności zadań



In [1]:
file = "witi.data.txt"
def load_data(data_name):
    data = []
    with open(file, 'r') as f:
        append = False
        for line in f.readlines():
            if line.startswith("\n"):
                append = False
                
            if append:
                s = line.strip()
                s = s.split()
                if len(s) <= 1:
                    continue
                
                data.append([int(s[0]), int(s[1]), int(s[2])])
            if line.startswith(data_name):
                append = True

    return data
print(load_data("data.10"))

[[1, 2, 748], [46, 5, 216], [5, 7, 673], [93, 4, 514], [83, 1, 52], [53, 7, 7], [38, 1, 413], [68, 6, 922], [84, 5, 91], [65, 4, 694]]


In [2]:
data_names = []
for i in range(10, 21):
    data_names.append("data." + str(i))
print(data_names)

['data.10', 'data.11', 'data.12', 'data.13', 'data.14', 'data.15', 'data.16', 'data.17', 'data.18', 'data.19', 'data.20']


In [3]:
def calculate_raw(data):
    penalty = 0
    time = 0

    for i in range(len(data)):
        time += data[i][0]  # dodaj czas trwania zadania
        delay = max(0, time - data[i][2])  # opóźnienie = (czas zakończenia - deadline) lub 0
        penalty += data[i][1] * delay  # kara: waga * opóźnienie
    return penalty
print(calculate_raw(load_data("data.11")))
        

6323


In [4]:
import math

def witi_pd(data):
    n = len(data)
    dp = [math.inf] * (1 << n)  # inf dla wszystkich masek
    dp[0] = 0  # dla pustego zbioru kara = 0
    C = [0] * (1 << n)  # czas wykonania wszystkich zadań w masce
    prev = [-1] * (1 << n)  # do odtworzenia kolejności
    
    # Precomputing C[mask]
    for mask in range(1 << n):
        for i in range(n):
            if (mask >> i) & 1:
                C[mask] += data[i][0]  # czas produkcji i-tego zadania

    # Dynamiczne programowanie
    for mask in range(1 << n):
        for i in range(n):
            if (mask >> i) & 1:  # jeśli zadanie i jest w masce
                prev_mask = mask ^ (1 << i)  # usuwamy zadanie i
                finish_time = C[mask]
                delay = max(0, finish_time - data[i][2])  # opóźnienie
                penalty = data[i][1] * delay
                if dp[mask] > dp[prev_mask] + penalty:
                    dp[mask] = dp[prev_mask] + penalty
                    prev[mask] = i  # zapamiętaj co wybraliśmy ostatnio
    
    # Odtwarzanie kolejności
    mask = (1 << n) - 1  # wszystkie zadania
    order = []
    while mask:
        task = prev[mask]
        order.append(task)
        mask = mask ^ (1 << task)

    order = order[::-1]  # odwrócenie kolejności, bo dodawaliśmy od końca

    return dp[(1 << n) - 1], order  # zwracamy minimalną karę i kolejność zadań

# Przykład użycia:
data = load_data("data.11")
penalty, order = witi_pd(data)
print("Minimalna kara:", penalty)
print("Optymalna kolejność zadań:", order)


Minimalna kara: 799
Optymalna kolejność zadań: [5, 8, 1, 10, 4, 6, 9, 3, 7, 2, 0]
